# Sprint 5 · Webinar 21 · Data Analytics teórico (Outliers + GitHub para análisis)

En esta sesión profundizaremos en **detección y tratamiento de outliers** (valores atípicos) con reglas estadísticas y criterio de negocio. También construiremos **segmentos de clientes de forma programática** y practicaremos cómo **almacenar y compartir análisis** con un flujo de trabajo en GitHub, incluyendo un ejemplo aplicable en Google Colab.

**Duración estimada:** 100 minutos

## Fecha

- **Fecha:** 2025-12-14
- **Modalidad:** Sincrónica (teórica con demostraciones cortas)
- **Herramientas:** Python (Pandas, NumPy, Matplotlib), Git, GitHub, Google Colab

## Objetivos de la sesión teórica

Al finalizar esta sesión, el/la estudiante podrá:

1. Explicar qué es un outlier y por qué **no siempre** es un error.
2. Identificar outliers usando reglas estadísticas (IQR, Z-score, percentiles y MAD).
3. Seleccionar una estrategia de tratamiento de outliers según **contexto de negocio**.
4. Crear segmentos de clientes de forma programática (ej. RFM) usando funciones y reglas.
5. Elaborar un **resumen estadístico interpretativo** (técnico + significado de negocio).
6. Aplicar conceptos básicos de GitHub para almacenar, estructurar y compartir análisis, y ejecutar un flujo de trabajo básico en Colab.

## Agenda sugerida (100 minutos)

| Bloque | Tema | Tiempo |
|---:|---|---:|
| 1 | Contexto: qué son outliers y por qué importan | 10 min |
| 2 | 7.3.1 Reglas estadísticas para detectar outliers | 25 min |
| 3 | 7.3.2 Tratamiento de outliers con criterio de negocio | 20 min |
| 4 | 7.3.3 Segmentación de clientes (programática) | 20 min |
| 5 | 7.3.4 Resumen estadístico e interpretación | 10 min |
| 6 | 7.4 GitHub para analistas: conceptos + estructura + Colab | 15 min |


## Ejercicio 0 · Calentamiento (discusión conceptual, 10 min)

**Instrucciones (breakout rooms):**

1. Den 2 ejemplos reales donde un outlier **sea un error** (dato inválido) y 2 donde un outlier **sea una señal** (evento real valioso).
2. En cada ejemplo, indiquen qué harían: **eliminar**, **corregir**, **capar**, **transformar**, **marcar** o **mantener**.
3. Definan una regla: ¿qué evidencia mínima exigirían para eliminar un dato?

**Output esperado:** lista corta de casos + decisión + justificación.

## Ejercicio 1 · Generar dataset y explorar posibles outliers

En esta sesión usaremos un dataset sintético de clientes (estilo e-commerce) con variables numéricas y categóricas.

**Objetivo:** crear una base reproducible, describirla y detectar *candidatos* a outliers.

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rng = np.random.default_rng(42)

n = 1200
customers = pd.DataFrame({
    "customer_id": np.arange(1, n+1),
    "region": rng.choice(["Norte", "Centro", "Sur"], size=n, p=[0.35, 0.40, 0.25]),
    "channel": rng.choice(["Web", "App", "Tienda"], size=n, p=[0.55, 0.35, 0.10]),
    "days_since_last": rng.integers(0, 365, size=n),
})

# Variables de negocio (con colas largas típicas)
customers["orders_90d"] = rng.poisson(lam=3.2, size=n)
base_spend = rng.lognormal(mean=4.2, sigma=0.55, size=n)   # gasto 90 días
customers["spend_90d"] = np.round(base_spend, 2)

# AOV aproximado: gasto / órdenes (evitar división por cero)
customers["avg_order_value"] = np.where(customers["orders_90d"] > 0,
                                        customers["spend_90d"] / customers["orders_90d"],
                                        0.0)
customers["avg_order_value"] = customers["avg_order_value"].round(2)

# Inyectar outliers: clientes con gasto y órdenes extremadamente altos
outlier_idx = rng.choice(customers.index, size=12, replace=False)
customers.loc[outlier_idx, "orders_90d"] = customers.loc[outlier_idx, "orders_90d"] + rng.integers(25, 60, size=12)
customers.loc[outlier_idx, "spend_90d"] = customers.loc[outlier_idx, "spend_90d"] * rng.uniform(8, 20, size=12)
customers["spend_90d"] = customers["spend_90d"].round(2)
customers["avg_order_value"] = np.where(customers["orders_90d"] > 0,
                                        customers["spend_90d"] / customers["orders_90d"],
                                        0.0).round(2)

customers.head()


### Preguntas guiadas para el Ejercicio 1

1. ¿Qué variables numéricas presentan colas largas (skew)?
2. ¿Cómo cambia la interpretación si segmentas por `channel` o `region`?
3. ¿Qué visual te ayuda más: histograma, boxplot o scatter?
4. ¿Un cliente con gasto alto es necesariamente un problema?


In [ ]:

# Estadísticas rápidas
customers[["orders_90d", "spend_90d", "avg_order_value", "days_since_last"]].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]).T


In [ ]:

# Visualización rápida: histograma y boxplot para gasto
customers["spend_90d"].plot(kind="hist", bins=40, title="Distribución de spend_90d")
plt.show()

customers[["spend_90d"]].plot(kind="box", title="Boxplot de spend_90d")
plt.show()


## Ejercicio 2 · 7.3 Detecting and Treating Outliers

### 7.3.1 Identifying Outliers with Statistical Rules

Reglas comunes (no exclusivas):

- **IQR (Tukey):** outliers por debajo de Q1 − 1.5·IQR o por encima de Q3 + 1.5·IQR.
- **Z-score:** distancia en desviaciones estándar (útil si distribución ~normal).
- **Percentiles:** capar o marcar extremos (ej. > P99 o < P1).
- **MAD / Modified Z-score:** robusto ante colas largas.

### 7.3.2 Handling Outliers in Context

Estrategias típicas (elige según el objetivo):

1. **Eliminar** (solo si hay evidencia fuerte de error o duplicidad).
2. **Capar/Winsorizar** (limita extremos manteniendo tamaño muestral).
3. **Transformar** (log, Box-Cox, Yeo-Johnson) para estabilizar varianza.
4. **Imputar/Corregir** (si proviene de una regla de validación).
5. **Marcar** (`is_outlier`) y modelar con esa señal.

**Objetivo del ejercicio:** implementar 2 detectores (IQR y MAD) y comparar decisiones de tratamiento.

In [ ]:

def outliers_iqr(s: pd.Series, k: float = 1.5):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    low, high = q1 - k * iqr, q3 + k * iqr
    mask = (s < low) | (s > high)
    return mask, (low, high)

def outliers_mad(s: pd.Series, thresh: float = 3.5):
    # Modified Z-score basado en MAD (robusto)
    med = s.median()
    mad = (s - med).abs().median()
    if mad == 0:
        return pd.Series(False, index=s.index), (med, mad)
    mz = 0.6745 * (s - med) / mad
    mask = mz.abs() > thresh
    return mask, (med, mad)

spend = customers["spend_90d"]

mask_iqr, (low_iqr, high_iqr) = outliers_iqr(spend)
mask_mad, (med, mad) = outliers_mad(spend)

pd.DataFrame({
    "method": ["IQR", "MAD"],
    "outliers_detected": [int(mask_iqr.sum()), int(mask_mad.sum())],
    "share": [mask_iqr.mean(), mask_mad.mean()],
    "thresholds": [f"[{low_iqr:.2f}, {high_iqr:.2f}]", f"median={med:.2f}, MAD={mad:.2f}"]
})


In [ ]:

customers = customers.copy()
customers["is_outlier_spend_iqr"] = mask_iqr
customers["is_outlier_spend_mad"] = mask_mad

# Comparación de los outliers detectados
both = (customers["is_outlier_spend_iqr"] & customers["is_outlier_spend_mad"]).sum()
only_iqr = (customers["is_outlier_spend_iqr"] & ~customers["is_outlier_spend_mad"]).sum()
only_mad = (~customers["is_outlier_spend_iqr"] & customers["is_outlier_spend_mad"]).sum()
{"both": int(both), "only_iqr": int(only_iqr), "only_mad": int(only_mad)}


In [ ]:

# Tratamiento ejemplo: winsorización por percentil 1-99 (capado)
p01, p99 = customers["spend_90d"].quantile([0.01, 0.99])
customers["spend_90d_winsor"] = customers["spend_90d"].clip(lower=p01, upper=p99)

# Tratamiento ejemplo: transformación log (cuidado con ceros)
customers["spend_90d_log"] = np.log1p(customers["spend_90d"])

customers[["spend_90d", "spend_90d_winsor", "spend_90d_log"]].describe().T


## Ejercicio 3 · Segmentación programática + resumen estadístico + GitHub

### 7.3.3 Creating Customer Segments Programmatically

Crearemos segmentos tipo **RFM** simplificado:

- **R (Recency):** `days_since_last` (menor = mejor)
- **F (Frequency):** `orders_90d`
- **M (Monetary):** `spend_90d_winsor`

### 7.3.4 Writing a Statistical Summary

Redactaremos un resumen con:
- tamaño de muestra, distribución, missing/invalid (si aplica), outliers detectados
- insights por segmento (medianas/percentiles)
- implicaciones de negocio (acciones)

### 7.4 Store and Share Analysis

Introducción a GitHub para analistas: valor, estructura de repos y un flujo en Colab.

In [ ]:

# Puntajes RFM por cuantiles (1 a 4)
def score_quantiles(s: pd.Series, q=4, higher_is_better=True):
    # Devuelve scores 1..q
    ranks = pd.qcut(s.rank(method="first"), q=q, labels=False) + 1
    if higher_is_better:
        return ranks
    else:
        return (q + 1) - ranks

customers["R_score"] = score_quantiles(customers["days_since_last"], q=4, higher_is_better=False)
customers["F_score"] = score_quantiles(customers["orders_90d"], q=4, higher_is_better=True)
customers["M_score"] = score_quantiles(customers["spend_90d_winsor"], q=4, higher_is_better=True)

customers["RFM"] = customers["R_score"].astype(str) + customers["F_score"].astype(str) + customers["M_score"].astype(str)

# Regla simple de segmentación (ejemplo)
def segment(r, f, m):
    if (r >= 3) and (f >= 3) and (m >= 3):
        return "Champions"
    if (r >= 3) and (f >= 2) and (m >= 2):
        return "Loyal"
    if (r <= 2) and (f >= 3):
        return "At Risk (Freq)"
    if (r <= 2) and (m >= 3):
        return "At Risk (Value)"
    if (r == 1) and (f == 1) and (m == 1):
        return "Hibernating"
    return "Potential"

customers["segment"] = customers.apply(lambda row: segment(row["R_score"], row["F_score"], row["M_score"]), axis=1)
customers["segment"].value_counts()


In [ ]:

# Resumen por segmento (medianas + tamaño)
seg_summary = (customers
    .groupby("segment")
    .agg(
        n=("customer_id", "count"),
        spend_median=("spend_90d_winsor", "median"),
        spend_p90=("spend_90d_winsor", lambda x: x.quantile(0.90)),
        orders_median=("orders_90d", "median"),
        recency_median=("days_since_last", "median"),
        outlier_rate=("is_outlier_spend_mad", "mean"),
    )
    .sort_values("n", ascending=False)
)
seg_summary


In [ ]:

# Visual: boxplot de gasto por segmento
customers.boxplot(column="spend_90d_winsor", by="segment", rot=45)
plt.title("Spend_90d (winsor) por segmento")
plt.suptitle("")
plt.ylabel("spend_90d_winsor")
plt.show()


In [ ]:

# Redacción programática de un resumen estadístico básico (plantilla)
def write_stat_summary(df: pd.DataFrame) -> str:
    n = len(df)
    outliers = int(df["is_outlier_spend_mad"].sum())
    outlier_rate = df["is_outlier_spend_mad"].mean()
    p50 = df["spend_90d_winsor"].median()
    p90 = df["spend_90d_winsor"].quantile(0.90)
    p99 = df["spend_90d_winsor"].quantile(0.99)

    top_segment = df["segment"].value_counts().index[0]
    top_share = df["segment"].value_counts(normalize=True).iloc[0]

    lines = []
    lines.append(f"Resumen del dataset (n={n} clientes):")
    lines.append(f"- Outliers (MAD) en spend_90d: {outliers} ({outlier_rate:.1%}).")
    lines.append(f"- spend_90d_winsor: mediana={p50:.2f}, P90={p90:.2f}, P99={p99:.2f}.")
    lines.append(f"- Segmento más frecuente: {top_segment} ({top_share:.1%}).")
    lines.append("")
    lines.append("Interpretación (ejemplo):")
    lines.append("- Si la tasa de outliers es baja pero concentra alto valor, conviene marcarlos y analizarlos como clientes VIP/fraude según contexto.")
    lines.append("- La diferencia entre P90 y P99 sugiere cola larga; para reportes agregados, usar percentiles/medianas puede ser más robusto que el promedio.")
    return "\n".join(lines)

print(write_stat_summary(customers))


## 6. Take aways de la sesión teórica

- Un outlier es un **dato extremo**; puede ser error, fraude o un cliente valioso: la decisión correcta depende del **contexto**.
- IQR y MAD suelen ser más robustos que Z-score cuando hay colas largas.
- Antes de eliminar, prioriza: **validaciones**, **marcado** (`is_outlier`) y **tratamientos conservadores** (winsor/log).
- Segmentación programática (RFM) es una forma práctica de traducir distribuciones en **acciones**.
- Un buen resumen estadístico combina métricas (P50/P90/P99) con interpretación y recomendación.
- GitHub no es solo para desarrolladores: permite **versionar**, **compartir**, **reproducir** y **auditar** análisis.

## 7. Cierre y próximos pasos

**Checklist de práctica sugerida (post-clase):**

1. Ajusta el umbral de MAD (ej. 3.0 vs 4.0) y observa impacto en tasa de outliers.
2. Compara decisiones: eliminar vs winsor vs log y evalúa cómo cambian medianas/percentiles.
3. Redefine reglas de segmentación (más/menos estrictas) y valida estabilidad del tamaño por segmento.
4. Redacta un resumen final de 8–10 líneas para un stakeholder (no técnico).
5. Sube el notebook a un repositorio (GitHub) con una estructura mínima y un README.

## 8. Información complementaria y recursos

### 7.4.1 GitHub for Analysts: Concepts & Value

- **Repositorio:** carpeta versionada con historial (commits).
- **Commit:** “foto” del estado del proyecto con mensaje.
- **Branch:** línea paralela de trabajo (útil para experimentar).
- **Pull request:** mecanismo para revisar y fusionar cambios.

### 7.4.2 Repos & Project Structure (hands-on setup)

Estructura mínima recomendada:

```
my-analysis/
  data/            # (opcional) datos pequeños o muestras
  notebooks/       # notebooks (.ipynb)
  src/             # funciones reutilizables (.py)
  reports/         # salidas, figuras
  README.md
  requirements.txt
```

Buenas prácticas:
- No subir datos sensibles; usar `.gitignore`.
- Mensajes de commit: cortos y específicos (ej. `add MAD outlier detector`).

### 7.4.3 Applying the GitHub Workflow with Google Colab

**Flujo típico en Colab (conceptual):**

1. Clonar repo: `!git clone <url>`
2. Trabajar en `notebooks/` o `src/`
3. `git status` → `git add` → `git commit -m "..."`
4. `git push` (autenticación con token)

Nota: en Colab, la autenticación suele requerir un **Personal Access Token** y configurar usuario/email local.